Collatz vertex notebook

In [1]:
"""
This notebook tests a formula that calculates a specific vertex (odd number)
of a Collatz sequence following a specific path. The formula models a path 
where the sequence initially decreases and subsequently enters the worst case path
in only one division by 2 takes place in every iteration.
"""

# Imports
from math import log2
import pandas as pd
from collatz import generator as gen
from collatz import commons as com

# Configuration
K_FACTOR = 3
MAX_ITERATIONS = 50
PRINT_TABLE = True

# Create a starting value that leads to the analysed path
# START_VALUE = int('11111110101', 2)
START_VALUE = 805675

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.expand_frame_repr', False)

# Create a collatz sequence and analyse it
analysis_frame = gen.generate_collatz_sequence(
    start_value=START_VALUE, k=K_FACTOR, max_iterations=MAX_ITERATIONS)

# Filter odd values
analysis_frame = analysis_frame[analysis_frame["odd"] == 1]
analysis_frame = analysis_frame.reset_index(drop=True)

# Derive new fields
analysis_frame["v1"] = START_VALUE
analysis_frame["n"] = analysis_frame.index + 1

analysis_frame["alpha"] = analysis_frame["next_collatz"].apply(com.trailing_zeros)
analysis_frame["alpha"] = analysis_frame["alpha"].astype("int64")
analysis_frame["alpha_sum"] = analysis_frame["alpha"].cumsum()
analysis_frame["alpha_pred"] = (log2(K_FACTOR) * analysis_frame["n"]).astype('int64') + 1
analysis_frame["alpha_diff"] = analysis_frame["alpha_sum"] - analysis_frame["alpha_pred"]

analysis_frame["alpha1"] = analysis_frame["alpha"][0]
analysis_frame["alpha2"] = analysis_frame["alpha_sum"] - analysis_frame["alpha1"]

analysis_frame["next_odd_pred"] = 3**(analysis_frame["alpha2"] + 1) * analysis_frame["v1"]
analysis_frame["next_odd_pred"] = analysis_frame["next_odd_pred"] + 3**analysis_frame["alpha2"]
analysis_frame["next_odd_pred"] = analysis_frame["next_odd_pred"] + \
                         (3**(analysis_frame["alpha2"]) * 2**(analysis_frame["alpha1"]))

analysis_frame["next_odd_pred"] = (analysis_frame["next_odd_pred"] / 
                                  2**(analysis_frame["alpha_sum"])) - 1

analysis_frame["next_odd_pred"] = analysis_frame["next_odd_pred"].astype('int64')
                         
print_frame = analysis_frame[[
    "n", "v1", "collatz", "next_odd", "next_odd_pred", "alpha_sum", "alpha_pred", 
    "alpha1", "alpha2", "bin_str"]]

print_frame.columns = [
    "n","v1", "vn", "vn+1", "vn+1_pred", "a", "a_pred", "a1", "a2", "bin"]

print("Start value:", START_VALUE, 
      " K:", K_FACTOR, "\n")

if PRINT_TABLE:
    print(print_frame.to_string(index=False), "\n")

Start value: 805675  K: 3 

  n      v1       vn     vn+1    vn+1_pred   a  a_pred  a1  a2                    bin
  1  805675   805675  1208513      1208513   1       2   1   0   11000100101100101011
  2  805675  1208513   906385      2719155   3       4   1   2  100100111000011000001
  3  805675   906385   679789      6118101   5       5   1   4   11011101010010010001
  4  805675   679789   254921     20648593   8       7   1   7   10100101111101101101
  5  805675   254921   191191     46459337  10       8   1   9     111110001111001001
  6  805675   191191   286787     69689006  11      10   1  10     101110101011010111
  7  805675   286787   430181    104533509  12      12   1  11    1000110000001000011
  8  805675   430181    80659    529200896  16      13   1  15    1101001000001100101
  9  805675    80659   120989    793801344  17      15   1  16      10011101100010011
 10  805675   120989    45371   2679079540  20      16   1  19      11101100010011101
 11  805675    45371    68